# Extract BERT

Notebook to experiment with libraries to extract parts of BERT embeddings

In [2]:
import transformers
import torch

I1027 15:12:34.159069 140375016859392 file_utils.py:39] PyTorch version 1.2.0 available.
I1027 15:12:35.182094 140375016859392 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [ ]:
bert_model = transformers.BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
bert_tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
input_ids = bert_tokenizer.encode('My cat is called Xiaonuanhuo and she is warm and fluffy')
input_ids

[2026,
 4937,
 2003,
 2170,
 19523,
 11231,
 2319,
 6979,
 2080,
 1998,
 2016,
 2003,
 4010,
 1998,
 27036]

In [11]:
len(input_ids)

15

In [12]:
bert_embeddings = bert_model(torch.tensor([input_ids]))
bert_embeddings[0]

tensor([[[-0.4095,  0.0730,  0.0156,  ...,  0.0193,  0.1457,  0.6264],
         [-0.6220, -0.3098,  0.4594,  ..., -0.2029,  0.3284,  1.0646],
         [-1.0167, -0.3767,  0.4307,  ..., -0.0539,  0.4236,  0.9048],
         ...,
         [-0.7988,  0.2165,  0.7978,  ..., -0.0076,  0.0286,  0.4517],
         [-0.7674, -0.0174,  0.5640,  ...,  0.0461,  0.0452,  0.5110],
         [-0.7672,  0.1120,  0.8036,  ...,  0.1290,  0.1001,  0.5393]]],
       grad_fn=<NativeLayerNormBackward>)

In [13]:
# Final hidden layer, one for each token
bert_embeddings[0].shape

torch.Size([1, 15, 768])

In [14]:
# Pooled layer
bert_embeddings[1].shape

torch.Size([1, 768])

In [16]:
len(bert_embeddings[2])

13

In [19]:
# Nth hidden layer of BERT
N=7
bert_embeddings[2][N].shape

torch.Size([1, 15, 768])